In [ ]:
from specsanalyzer import SpecsAnalyzer
import matplotlib.pyplot as plt

In [ ]:
spa = SpecsAnalyzer()

In [ ]:
print(spa)

In [ ]:
import numpy as np
ar = np.ones((10,10), float)
spa.convert_image(ar, 15, 10, "WideAngleMode")

In [ ]:
import pandas as pd
tsv_data = pd.read_csv('../tests/data/dataFHI/2017/03 March/Day 09/Raw Data/1232/AVG/000.tsv', sep='\t')
plt.imshow(tsv_data)

In [ ]:
spa = SpecsAnalyzer(config="../tests/data/config/config.yaml")

In [ ]:
print(spa)

In [ ]:
spa.convert_image(tsv_data, 15, 10, 1, 2, {})

In [ ]:
import xarray as xr
import numpy as np
angle_axis = np.linspace(0,1,tsv_data.shape[0])
ek_axis = np.linspace(0,1,tsv_data.shape[1])
da = xr.DataArray(
    data=tsv_data,
    coords={"Angle": angle_axis, "Ekin": ek_axis},
    dims=["Angle", "Ekin"],
)
da.plot()

In [ ]:
from specsanalyzer.img_tools import crop_xarray
da_cropped = crop_xarray(da, 0.1, 0.9, 0.3, 0.9)
da_cropped.T.plot()

In [ ]:
# Testing convert and crop on the imported data
res_xarray = spa.convert_image(tsv_data, 15, 10, "WideAngleMode")
res_xarray.plot()

In [ ]:
res_cropped = crop_xarray(res_xarray, -5, 5, 9.5, 10)
res_cropped.T.plot()